Per últim separem per les cuatre aules de les quals tenim a disposició notes d'exàmens, i esborrem del general les que no 

In [49]:


# Define the aula_id values to filter
aula_ids = [87, 92, 143, 141]

# Loop through each aula_id, filter, and save as a new CSV file
for aula_id in aula_ids:
    # Filter activitats for the current aula_id
    filtered_df = activitats_filtered[activitats_filtered['aula_id'] == aula_id]
    
    # Save the filtered DataFrame to a CSV file
    filtered_df.to_csv(f'activitats_aula_{aula_id}.csv', index=False)

print("CSV files created for each specified aula_id.")


In [45]:
import pandas as pd
import spark
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import numpy as np


In [46]:
#read the data

activitats = pd.read_csv('data/activitats.csv')
notes = pd.read_csv('data/notes.csv')
trameses = pd.read_csv('data/trameses.csv')

#show the columns

print(activitats.columns)

#show the first 5 rows

print(activitats.head())



Index(['activitat_id', 'activitat', 'aula_id', 'startdate', 'duedate',
       'grade'],
      dtype='object')
   activitat_id                                activitat  aula_id  startdate  \
0             3            Problema 1.1 - Hello world!!!       87          0   
1             4         Problema 1.2 - Hello world!!! ++       87          0   
2             5               Problema 3a.1: C�lcul edat       87          0   
3             6  Problema 3a.8: Conversi� d�lars a euros       87          0   
4             7         Problema 3a.11: Mitjana de notes       87          0   

   duedate  grade  
0        0    100  
1        0    100  
2        0    100  
3        0    100  
4        0    100  


Comencem treient totes les activitats que no tenen cap tramesa associada


In [47]:

# Filter activitats to keep only entries with an activitat_id present in trameses
activitats_filtered = activitats[activitats['activitat_id'].isin(trameses['activitat_id'].unique())]

# Save or display the filtered activitats DataFrame
activitats_filtered.to_csv('activitats_filtered.csv', index=False)
print(activitats_filtered.head())


   activitat_id                                activitat  aula_id  startdate  \
0             3            Problema 1.1 - Hello world!!!       87          0   
1             4         Problema 1.2 - Hello world!!! ++       87          0   
2             5               Problema 3a.1: C�lcul edat       87          0   
3             6  Problema 3a.8: Conversi� d�lars a euros       87          0   
4             7         Problema 3a.11: Mitjana de notes       87          0   

   duedate  grade  
0        0    100  
1        0    100  
2        0    100  
3        0    100  
4        0    100  


A continuació afegim una clau shared_id que s'encarrega de relacionar tasques identiques que es troben a cursos diferents les que

In [48]:
# Create a dictionary to map each unique activitat to a unique ID
activitat_unique_id = {activitats_filtered: idx for idx, activitats_filtered in enumerate(activitats_filtered['activitat'].unique(), start=1)}

# Map the activitat values to the unique IDs in the pastas column
activitats_filtered['shared_id'] = activitats_filtered['activitat'].map(activitat_unique_id)

# Save or display the updated activitats DataFrame
activitats_filtered.to_csv('activitats_with_shared.csv', index=False)

/tmp/ipykernel_24364/3240484575.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activitats_filtered['shared_id'] = activitats_filtered['activitat'].map(activitat_unique_id)
